In [1]:
import pandas as pd
from pandas import json_normalize
# If you have a JSON string, you can convert it into a dictionary first
import json
with open('data.json', 'r') as f:
    data = json.load(f)

# Use json_normalize to convert nested JSON to a flat table
flat_data = json_normalize(data, record_path=['data'])
flat_data



,content,id,ref,type,bab,bagian,paragraf,pasal,level,context,additional_context,chunks,source_token_length,buku,alias,term
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BERITA NEGARA REPUBLIK INDONESIA No.920, 2017 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PERATURAN MENTERI KEUANGAN REPUBLIK INDONESIA ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DENGAN RAHMAT TUHAN YANG MAHA ESA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"MENTERI KEUANGAN REPUBLIK INDONESIA,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,SRI MULYANI INDRAWATI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2343,Diundangkan di Jakarta pada tanggal 7 Juli 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,DIREKTUR JENDERAL PERATURAN PERUNDANG-UNDANGAN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2345,ttd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# head data of each column that is not nan 
flat_data.head(10).dropna(axis=1, how='all')

,content,id,ref,type,bab,pasal,level
5,Menimbang :,heading_menimbang,heading_menimbang,konsideran,None,None,1.0
7,Mengingat :,heading_mengingat,heading_mengingat,konsideran,None,None,1.0
9,MEMUTUSKAN:,heading_memutuskan,heading_memutuskan,konsideran,None,None,1.0
11,"KEDUDUKAN, TUGAS, DAN FUNGSI",bab-i,BAB I,heading_bab,None,None,1.0
12,,pasal-1,Pasal 1,heading_pasal,bab-i,pasal-1,4.0
14,,pasal-2,Pasal 2,heading_pasal,bab-i,pasal-2,4.0
16,,pasal-3,Pasal 3,heading_pasal,bab-i,pasal-3,4.0
18,SUSUNAN ORGANISASI,bab-ii,BAB II,heading_bab,None,None,1.0
19,,pasal-4,Pasal 4,heading_pasal,bab-ii,pasal-4,4.0
21,"DIREKTUR KEUANGAN, UMUM, DAN SISTEM INFORMASI",bab-iii,BAB III,heading_bab,None,None,1.0
